# Get Started With nGQL Jupyter Magic Extension

## How To Install nGQL in Jupyter or iPython
First, install nGQL Magic with pip:
```bash
!pip install ipython-ngql
```
Second, load extension:
```bash
$load_ext ngql
```

In [1]:
%load_ext ngql

## How to Use
Connect to Nebula Graph with:

```bash
%ngql --address <ip> --port <port> --user <username> --password <password>
```

In [3]:
%ngql --address 192.168.8.137 --port 9669 --user root --password nebula

Connection Pool Created


,Name
0,auto_manufacturing_supply_chain
1,basketballplayer
2,demo_basketballplayer
3,demo_movie_recommendation
4,demo_sns
5,test


Make a query after `Connection Pool Created` shown in the connection from last step:


Option 1, it supports one line query as:
```ngql
%ngql <query_line>;
```
Option 2, it also supports multiple queries:

```ngql
%%ngql
<line 0>;
<line 1>;
```

> Note, we will support a 3rd option to enable runing lines from `.ngql` file soon.

In [4]:
%ngql USE basketballplayer;
%ngql MATCH (v:player{name:"Tim Duncan"})-->(v2:player) RETURN v2.player.name AS Name;

,Name
0,Tony Parker
1,Manu Ginobili


In [5]:
%%ngql
SHOW TAGS;
SHOW HOSTS;

,Host,Port,Status,Leader count,Leader distribution,Partition distribution,Version
0,storaged0,9779,ONLINE,8,"basketballplayer:3, demo_basketballplayer:3, d...","basketballplayer:3, demo_basketballplayer:3, d...",3.6.0
1,storaged1,9779,ONLINE,8,"basketballplayer:3, demo_basketballplayer:3, d...","basketballplayer:3, demo_basketballplayer:3, d...",3.6.0
2,storaged2,9779,ONLINE,12,"auto_manufacturing_supply_chain:1, basketballp...","auto_manufacturing_supply_chain:1, basketballp...",3.6.0


> It's quite easy, right?
You could get help by `%ngql help` for some advanced usage:

```ngql
%ngql help
```

### Using Variables in Query String

We used Jinja2(https://jinja.palletsprojects.com/) as templating method for variables in query string:

```python
trainer = "Sue"
```

```ngql
%%ngql
GO FROM "{{ trainer }}" OVER owns_pokemon YIELD owns_pokemon._dst as pokemon_id | GO FROM $-.pokemon_id OVER owns_pokemon REVERSELY YIELD owns_pokemon._dst AS Trainer_Name;
```

In [5]:
vid = "player100"

In [6]:
%%ngql
MATCH (v)<-[e:follow]- (v2)-[e2:serve]->(v3)
  WHERE id(v) == "{{ vid }}"
RETURN v2.player.name AS FriendOf, v3.team.name AS Team LIMIT 3;

,FriendOf,Team
0,LaMarcus Aldridge,Trail Blazers
1,LaMarcus Aldridge,Spurs
2,Marco Belinelli,Warriors


### Using Raw thrift data type as result
By default the result `ngql_result_style` is `pandas`, this enabled us to have table view renderred by Jupyter Notebook.

While, if you would like to get raw results from `neutron2-python` itself, just configure it as below on the fly:

```
%config IPythonNGQL.ngql_result_style="raw"
```

And after querying, the result will be stored in `_`, plesae then refer it to a new variable for further ad-hoc tweaking on it like:
```
$ngql <query>;

result = _

dir(result)
```

In [7]:
%config IPythonNGQL.ngql_result_style="raw"

In [8]:
%%ngql USE basketballplayer;
GO 2 STEPS FROM "player102" OVER follow YIELD dst(edge);

ResultSet(keys: ['dst(EDGE)'], values: ["player101"],["player125"],["player100"],["player102"],["player125"])

In [9]:
r = _

In [10]:
r.column_values("dst(EDGE)")[0].cast()

'player101'

> Change back to `pandas` `ngql_result_style`

In [11]:
%config IPythonNGQL.ngql_result_style="pandas"

In [12]:
%%ngql
GO FROM "player100", "player102" OVER serve \
  WHERE properties(edge).start_year > 1995 \
YIELD DISTINCT properties($$).name AS team_name, properties(edge).start_year AS start_year, properties($^).name AS player_name;

,team_name,start_year,player_name
0,Spurs,1997,Tim Duncan
1,Trail Blazers,2006,LaMarcus Aldridge
2,Spurs,2015,LaMarcus Aldridge


# Draw nGQL queries

> This depends on pyvis

In [ ]:
%pip install pyvis

In [12]:
%ngql match p=(v:player) WHERE id(v) == "player1000" return p LIMIT 5

,p
0,"(""player1000"" :player{age: 11, name: ""Jerry © ..."


In [13]:
%ng_draw

<class 'pyvis.network.Network'> |N|=1 |E|=0

In [22]:
%ngql GET SUBGRAPH 2 STEPS FROM "player101" YIELD VERTICES AS nodes, EDGES AS relationships;

,nodes,relationships
0,"[(""player101"" :player{})]","[(""player101"")-[:serve@0{}]->(""team204""), (""pl..."
1,"[(""team215"" :team{}), (""player113"" :player{}),...","[(""player111"")-[:serve@0{}]->(""team215""), (""pl..."
2,"[(""player146"" :player{}), (""team206"" :team{}),...","[(""player146"")-[:serve@0{}]->(""team222""), (""pl..."


In [23]:
%ng_draw

nebulagraph_draw.html


## Only one takeaway: `% ngql help`!
All you have to remember is to use `$ngql help` to have all hints :-)

In [19]:
%ngql help



        Supported Configurations:
        ------------------------
        
        > How to config ngql_result_style in "raw", "pandas"
        %config IPythonNGQL.ngql_result_style="raw"
        %config IPythonNGQL.ngql_result_style="pandas"

        > How to config ngql_verbose in True, False
        %config IPythonNGQL.ngql_verbose=True

        > How to config max_connection_pool_size
        %config IPythonNGQL.max_connection_pool_size=10

        Quick Start:
        -----------

        > Connect to Neubla Graph
        %ngql --address 127.0.0.1 --port 9669 --user user --password password

        > Use Space
        %ngql USE nba

        > Query
        %ngql SHOW TAGS;

        > Multile Queries
        %%ngql
        SHOW TAGS;
        SHOW HOSTS;

        Reload ngql Magic
        %reload_ext ngql

        > Variables in query, we are using Jinja2 here
        name = "nba"
        %ngql USE "{{ name }}"

        > Query and draw the graph

        %ngql GET SUBGRAPH 2 ST